<a href="https://colab.research.google.com/github/zolfaShefreie/MCI-Chatbot/blob/main/using_vosk_for_persian_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#installation

In [1]:
!pip install vosk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 8.2 MB/s 
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=2d63bdf036a4a04cd0aa45abfbcb9803b79ab70665ea9fc77b85e979384f0cf3
  Stored in directory: /root/.cache/pip/wheels/54/c4/ec/4604122e072aebb16803c8297b7cd3f4c72073a3ee58738015
Successfully built srt


In [2]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# get Model
download model and make KaldiRecognizer object

In [3]:
from vosk import Model, KaldiRecognizer, list_languages

In [4]:
print("Supported languages:")
list_languages()

Supported languages:
de
fr
nl
pl
sv
en-us
ru
tl-ph
it
cn
pt
tr
vn
hi
el-gr
ja
ar
en-in
ua
eo
kz
es
ca
fa
en-gb
cs


##download model
if it is downloaded before, don't run these cells<br/>
you can get model links from [here](https://alphacephei.com/vosk/models)

In [5]:
!wget https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip

--2022-09-11 07:16:56--  https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1130835264 (1.1G) [application/zip]
Saving to: ‘vosk-model-fa-0.5.zip’

vosk-model-fa-0.5.z 100%[===================>]   1.05G  19.3MB/s    in 57s     

2022-09-11 07:17:54 (18.8 MB/s) - ‘vosk-model-fa-0.5.zip’ saved [1130835264/1130835264]



In [6]:
!unzip "./vosk-model-fa-0.5.zip" -d "./"

Archive:  ./vosk-model-fa-0.5.zip
   creating: ./vosk-model-fa-0.5/
   creating: ./vosk-model-fa-0.5/rescore/
  inflating: ./vosk-model-fa-0.5/rescore/G.carpa  
  inflating: ./vosk-model-fa-0.5/rescore/G.fst  
   creating: ./vosk-model-fa-0.5/test/
 extracting: ./vosk-model-fa-0.5/test/test.txt  
  inflating: ./vosk-model-fa-0.5/test/test.wav  
  inflating: ./vosk-model-fa-0.5/test/test_simple.py  
   creating: ./vosk-model-fa-0.5/am/
  inflating: ./vosk-model-fa-0.5/am/final.mdl  
  inflating: ./vosk-model-fa-0.5/am/tree  
   creating: ./vosk-model-fa-0.5/graph/
  inflating: ./vosk-model-fa-0.5/graph/words.txt  
  inflating: ./vosk-model-fa-0.5/graph/HCLG.fst  
   creating: ./vosk-model-fa-0.5/graph/phones/
  inflating: ./vosk-model-fa-0.5/graph/phones/word_boundary.int  
   creating: ./vosk-model-fa-0.5/conf/
  inflating: ./vosk-model-fa-0.5/conf/model.conf  
  inflating: ./vosk-model-fa-0.5/conf/mfcc.conf  
   creating: ./vosk-model-fa-0.5/ivector/
  inflating: ./vosk-model-fa-0.5/i

##load model

In [11]:
model_path = "./vosk-model-fa-0.5"
sr = 44100

In [12]:
model = Model(model_path=model_path)

In [13]:
rec = KaldiRecognizer(model, sr)

In [14]:
rec.SetWords(True)
rec.SetPartialWords(False)

#Record Sound and save it to wav file
if you want record you voice, run these cells

the source code is [here](https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb#scrollTo=H4rxNhsEpr-c)

In [26]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from scipy.io.wavfile import write

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Stop Recording";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the record..."
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio(save=True, file_path="./record.wav"):
    """
    run javascript and record sound by user system then save and play the recorded sound

    """
    display(HTML(AUDIO_HTML))
    # run the script
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    
    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True))
    output, err = process.communicate(input=binary)
    
    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    if save:
      with open(file_path, mode='bw') as f:
        f.write(riff)
    
    return audio, sr

In [33]:
audio, sr = get_audio()

#convert file to text

In [17]:
import wave
import IPython
import json

In [34]:
file_path = input("file_path:") or "./record.wav"
IPython.display.Audio(file_path)

file_path:


In [35]:
wf = wave.open(file_path, 'rb')
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        print(rec.Result())
    else:
        print(rec.PartialResult())

print(rec.FinalResult())

{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : "سلام"
}
{
  "partial" : "سلام"
}
{
  "partial" : "سلام"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "partial" : "سلام خوب هستید"
}
{
  "result" : [{
      "conf" : 1.000000,
      "end" : 12.185510,
      "start" : 